In [12]:
from pathlib import Path
from tbparse import SummaryReader
import torch
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import pickle

In [ ]:
# plt.ion()
# sns.set(font_scale=1, style="white")

In [15]:
df_file = 'dataframe.pd'
if False: # os.path.exists(df_file):
  with open(df_file, 'rb') as f:
    df = pickle.load(f)
else:
  param_list = ["tau_mem", "scale_grad", "width_grad", "optimizer", "rand_seed"] # "batch_size", "decoding_func", "encoding_dim", "hidden_dim", "learning_rate", "n_hidden_layers", "spike_threshold"
  dfs = []
  for root, dirs, files in os.walk(Path('lightning_logs/ssc')):
    results = None
    if 'grads.pt' in files and 'hparams.yaml' in files:
      results = SummaryReader(root, pivot=True)
      df = results.scalars
      hp = results.hparams
      # drop a few columns that we don't use
      df = df[df.columns[~df.columns.str.startswith("grad_2.0")]]
      df = df.drop(columns=['hp_metric', 'step', 'epoch'])
      # add hyperparameters to table
      df['method'] = root.split("/")[2].upper() # method name, exodus or slayer
      df['run'] = root.split("/")[-1] # we ran each experiment 3 times, so version_0, 1 or 2
      df['folder'] = root
      for param in param_list:
        df[param] = hp[param][0]
      dfs.append(df)
  df = pd.concat(dfs, ignore_index=True)
  df.to_pickle(df_file)

In [17]:
df = df[df.columns[~df.columns.str.startswith("grad_2.0")]]
df

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
0,SLAYER,version_5,lightning_logs/ssc/slayer/200.0/1.0/sgd/version_5,200.0,1.00,1.0,sgd,1234.0
1,SLAYER,version_3,lightning_logs/ssc/slayer/200.0/1.0/sgd/version_3,200.0,1.00,1.0,sgd,12.0
2,SLAYER,version_4,lightning_logs/ssc/slayer/200.0/1.0/sgd/version_4,200.0,1.00,1.0,sgd,123.0
3,SLAYER,version_5,lightning_logs/ssc/slayer/200.0/1.0/adam/versi...,200.0,1.00,1.0,adam,1234.0
4,SLAYER,version_3,lightning_logs/ssc/slayer/200.0/1.0/adam/versi...,200.0,1.00,1.0,adam,12.0
...,...,...,...,...,...,...,...,...
187,EXODUS,version_3,lightning_logs/ssc/exodus/50.0/0.01/sgd/version_3,50.0,0.01,1.0,sgd,12.0
188,EXODUS,version_4,lightning_logs/ssc/exodus/50.0/0.01/sgd/version_4,50.0,0.01,1.0,sgd,123.0
189,EXODUS,version_5,lightning_logs/ssc/exodus/50.0/0.01/adam/versi...,50.0,0.01,1.0,adam,1234.0
190,EXODUS,version_3,lightning_logs/ssc/exodus/50.0/0.01/adam/versi...,50.0,0.01,1.0,adam,12.0


In [31]:
def grad_correl(s0, s1, key):
    grads0 = torch.load(s0.folder + "/grads.pt")[key]
    grads1 = torch.load(s1.folder + "/grads.pt")[key]
    return (grads0 * grads1).sum() / torch.sqrt( (grads0**2).sum() * (grads1**2).sum() )

# Exodus

## Correlation by time constant for scale 1

In [26]:
data = df[(df.method=="EXODUS") & (df.scale_grad==1.0) & (df.width_grad==1.0) & (df.optimizer=="sgd") & (df.rand_seed==12)].copy()
data.loc[:, ["tau_mem"]] = data["tau_mem"].astype(int)
data = data.sort_values("tau_mem")
data

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
145,EXODUS,version_3,lightning_logs/ssc/exodus/20.0/1.0/sgd/version_3,20,1.0,1.0,sgd,12.0
169,EXODUS,version_3,lightning_logs/ssc/exodus/50.0/1.0/sgd/version_3,50,1.0,1.0,sgd,12.0
97,EXODUS,version_3,lightning_logs/ssc/exodus/200.0/1.0/sgd/version_3,200,1.0,1.0,sgd,12.0
121,EXODUS,version_3,lightning_logs/ssc/exodus/100000.0/1.0/sgd/ver...,100000,1.0,1.0,sgd,12.0


In [38]:
key = "network.0.0.weight"
correlations = dict()
for i, s0 in data.iterrows():
    correlations[s0.tau_mem] = {
        s1.tau_mem: grad_correl(s0, s1, key).item()
        for j, s1 in data.iterrows()
    }  

{20: {20: 1.0, 50: 0.967438817024231, 200: 0.9217857122421265, 100000: 0.9074544310569763}, 50: {20: 0.967438817024231, 50: 1.0, 200: 0.966520369052887, 100000: 0.9526156783103943}, 200: {20: 0.9217857122421265, 50: 0.966520369052887, 200: 1.0, 100000: 0.9687582850456238}, 100000: {20: 0.9074544310569763, 50: 0.9526156783103943, 200: 0.9687582850456238, 100000: 1.0}}


In [39]:
pd.DataFrame(correlations)

,20,50,200,100000
20,1.000000,0.967439,0.921786,0.907454
50,0.967439,1.000000,0.966520,0.952616
200,0.921786,0.966520,1.000000,0.968758
100000,0.907454,0.952616,0.968758,1.000000


## Correlation by scale for tau 20

In [40]:
data = df[(df.method=="EXODUS") & (df.tau_mem==20) & (df.width_grad==1.0) & (df.optimizer=="sgd") & (df.rand_seed==12)].copy()
data = data.sort_values("scale_grad")
data

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
163,EXODUS,version_3,lightning_logs/ssc/exodus/20.0/0.01/sgd/version_3,20.0,0.01,1.0,sgd,12.0
157,EXODUS,version_3,lightning_logs/ssc/exodus/20.0/0.1/sgd/version_3,20.0,0.10,1.0,sgd,12.0
145,EXODUS,version_3,lightning_logs/ssc/exodus/20.0/1.0/sgd/version_3,20.0,1.00,1.0,sgd,12.0
151,EXODUS,version_3,lightning_logs/ssc/exodus/20.0/2.0/sgd/version_3,20.0,2.00,1.0,sgd,12.0


In [42]:
key = "network.0.0.weight"
correlations = dict()
for i, s0 in data.iterrows():
    correlations[s0.scale_grad] = {
        s1.scale_grad: grad_correl(s0, s1, key).item()
        for j, s1 in data.iterrows()
    }  
pd.DataFrame(correlations)

,0.01,0.10,1.00,2.00
0.01,1.000000,0.981792,0.797510,0.642350
0.10,0.981792,1.000000,0.883357,0.730413
1.00,0.797510,0.883357,1.000000,0.894929
2.00,0.642350,0.730413,0.894929,1.000000


## Correlation by scale for tau 100000

In [43]:
data = df[(df.method=="EXODUS") & (df.tau_mem==100000) & (df.width_grad==1.0) & (df.optimizer=="sgd") & (df.rand_seed==12)].copy()
data = data.sort_values("scale_grad")
data

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
139,EXODUS,version_3,lightning_logs/ssc/exodus/100000.0/0.01/sgd/ve...,100000.0,0.01,1.0,sgd,12.0
133,EXODUS,version_3,lightning_logs/ssc/exodus/100000.0/0.1/sgd/ver...,100000.0,0.10,1.0,sgd,12.0
121,EXODUS,version_3,lightning_logs/ssc/exodus/100000.0/1.0/sgd/ver...,100000.0,1.00,1.0,sgd,12.0
127,EXODUS,version_3,lightning_logs/ssc/exodus/100000.0/2.0/sgd/ver...,100000.0,2.00,1.0,sgd,12.0


In [44]:
key = "network.0.0.weight"
correlations = dict()
for i, s0 in data.iterrows():
    correlations[s0.scale_grad] = {
        s1.scale_grad: grad_correl(s0, s1, key).item()
        for j, s1 in data.iterrows()
    }  
pd.DataFrame(correlations)

,0.01,0.10,1.00,2.00
0.01,1.000000,0.955185,0.729802,0.575589
0.10,0.955185,1.000000,0.853877,0.662543
1.00,0.729802,0.853877,1.000000,0.785696
2.00,0.575589,0.662543,0.785696,1.000000


## Correlation by width for tau 100000 and scale 1

In [46]:
data = df[(df.method=="EXODUS") & (df.tau_mem==100000) & (df.scale_grad==1.0) & (df.optimizer=="sgd") & (df.rand_seed==12)].copy()
data = data.sort_values("scale_grad")
data

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
121,EXODUS,version_3,lightning_logs/ssc/exodus/100000.0/1.0/sgd/ver...,100000.0,1.0,1.0,sgd,12.0


In [47]:
key = "network.0.0.weight"
correlations = dict()
for i, s0 in data.iterrows():
    correlations[s0.width_grad] = {
        s1.width_grad: grad_correl(s0, s1, key).item()
        for j, s1 in data.iterrows()
    }  
pd.DataFrame(correlations)

,1.0
1.0,1.0


# SLAYER

## Correlation by time constant for scale 1

In [48]:
data = df[(df.method=="SLAYER") & (df.scale_grad==1.0) & (df.width_grad==1.0) & (df.optimizer=="sgd") & (df.rand_seed==12)].copy()
data.loc[:, ["tau_mem"]] = data["tau_mem"].astype(int)
data = data.sort_values("tau_mem")
data

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
49,SLAYER,version_3,lightning_logs/ssc/slayer/20.0/1.0/sgd/version_3,20,1.0,1.0,sgd,12.0
73,SLAYER,version_3,lightning_logs/ssc/slayer/50.0/1.0/sgd/version_3,50,1.0,1.0,sgd,12.0
1,SLAYER,version_3,lightning_logs/ssc/slayer/200.0/1.0/sgd/version_3,200,1.0,1.0,sgd,12.0
25,SLAYER,version_3,lightning_logs/ssc/slayer/100000.0/1.0/sgd/ver...,100000,1.0,1.0,sgd,12.0


In [51]:
key = "linear_input.weight"
correlations = dict()
for i, s0 in data.iterrows():
    correlations[s0.tau_mem] = {
        s1.tau_mem: grad_correl(s0, s1, key).item()
        for j, s1 in data.iterrows()
    }  

In [52]:
pd.DataFrame(correlations)

,20,50,200,100000
20,1.000000,0.969953,0.939826,0.904281
50,0.969953,1.000000,0.986573,0.970476
200,0.939826,0.986573,1.000000,0.984845
100000,0.904281,0.970476,0.984845,1.000000


## Correlation by scale for tau 20

In [55]:
data = df[(df.method=="SLAYER") & (df.tau_mem==20) & (df.width_grad==1.0) & (df.optimizer=="sgd") & (df.rand_seed==12)].copy()
data = data.sort_values("scale_grad")
data

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
67,SLAYER,version_3,lightning_logs/ssc/slayer/20.0/0.01/sgd/version_3,20.0,0.01,1.0,sgd,12.0
61,SLAYER,version_3,lightning_logs/ssc/slayer/20.0/0.1/sgd/version_3,20.0,0.10,1.0,sgd,12.0
49,SLAYER,version_3,lightning_logs/ssc/slayer/20.0/1.0/sgd/version_3,20.0,1.00,1.0,sgd,12.0
55,SLAYER,version_3,lightning_logs/ssc/slayer/20.0/2.0/sgd/version_3,20.0,2.00,1.0,sgd,12.0


In [56]:
key = "linear_input.weight"
correlations = dict()
for i, s0 in data.iterrows():
    correlations[s0.scale_grad] = {
        s1.scale_grad: grad_correl(s0, s1, key).item()
        for j, s1 in data.iterrows()
    }  
pd.DataFrame(correlations)

,0.01,0.10,1.00,2.00
0.01,1.0,1.0,1.0,1.0
0.10,1.0,1.0,1.0,1.0
1.00,1.0,1.0,1.0,1.0
2.00,1.0,1.0,1.0,1.0


## Correlation by scale for tau 100000

In [57]:
data = df[(df.method=="SLAYER") & (df.tau_mem==100000) & (df.width_grad==1.0) & (df.optimizer=="sgd") & (df.rand_seed==12)].copy()
data = data.sort_values("scale_grad")
data

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
43,SLAYER,version_3,lightning_logs/ssc/slayer/100000.0/0.01/sgd/ve...,100000.0,0.01,1.0,sgd,12.0
37,SLAYER,version_3,lightning_logs/ssc/slayer/100000.0/0.1/sgd/ver...,100000.0,0.10,1.0,sgd,12.0
25,SLAYER,version_3,lightning_logs/ssc/slayer/100000.0/1.0/sgd/ver...,100000.0,1.00,1.0,sgd,12.0
31,SLAYER,version_3,lightning_logs/ssc/slayer/100000.0/2.0/sgd/ver...,100000.0,2.00,1.0,sgd,12.0


In [58]:
key = "linear_input.weight"
correlations = dict()
for i, s0 in data.iterrows():
    correlations[s0.scale_grad] = {
        s1.scale_grad: grad_correl(s0, s1, key).item()
        for j, s1 in data.iterrows()
    }  
pd.DataFrame(correlations)

,0.01,0.10,1.00,2.00
0.01,1.0,1.0,1.0,1.0
0.10,1.0,1.0,1.0,1.0
1.00,1.0,1.0,1.0,1.0
2.00,1.0,1.0,1.0,1.0


## Correlation by width for tau 100000 and scale 1

In [60]:
data = df[(df.method=="SLAYER") & (df.tau_mem==100000) & (df.scale_grad==1.0) & (df.optimizer=="sgd") & (df.rand_seed==12)].copy()
data = data.sort_values("scale_grad")
data

,method,run,folder,tau_mem,scale_grad,width_grad,optimizer,rand_seed
25,SLAYER,version_3,lightning_logs/ssc/slayer/100000.0/1.0/sgd/ver...,100000.0,1.0,1.0,sgd,12.0


In [61]:
key = "linear_input.weight"
correlations = dict()
for i, s0 in data.iterrows():
    correlations[s0.width_grad] = {
        s1.width_grad: grad_correl(s0, s1, key).item()
        for j, s1 in data.iterrows()
    }  
pd.DataFrame(correlations)

,1.0
1.0,1.0
